# Main Parsing

In [ ]:
!pip install -q demoparser2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 16.3 MB/s eta 0:00:00


In [ ]:
from demoparser2 import DemoParser
from uuid import uuid4
import numpy as np
import pandas as pd

parser = DemoParser("/content/drive/MyDrive/DemoAI/Demos/1-5adf10e8-f556-4c08-8017-b0f64c857e46-1-1.dem")
event_df = parser.parse_event("player_death", player=["X", "Y"], other=["total_rounds_played"])
info = parser.parse_header()
tick_df = parser.parse_ticks(
    ["X", "Y", "Z", "start_balance", "balance", "total_rounds_played", "is_match_started", "is_freeze_period", "game_time", "is_alive", "is_warmup_period", "team_name", "active_weapon_name"]
)

################# FIX OF PROBLEM THAT PREV ROUND DATA APPEARS IN FUTURE ROUND BEGIN DATA
tick_df = tick_df.sort_values(["total_rounds_played", "game_time"])

last_freeze_rows = (
    tick_df[tick_df['is_freeze_period'] == True]
    .groupby('total_rounds_played')['game_time']
    .max()
    .reset_index()
    .rename(columns={'game_time': 'last_freeze_time'})
)
# 2. добавляем это время в основной датасет
tick_df = tick_df.merge(last_freeze_rows, on='total_rounds_played', how='left')
# 3. удаляем все строки до и включая freeze-период
tick_df = tick_df[tick_df['game_time'] > tick_df['last_freeze_time']].drop(columns=['last_freeze_time'])

############## END FIX

tick_df['map_name'] = info['map_name']
tick_df['game_name'] = uuid4()
# разница с предыдущей точкой
tick_df['dx'] = tick_df['X'].diff()
tick_df['dy'] = tick_df['Y'].diff()

# длина вектора (модуль)
length = np.sqrt(tick_df['dx']**2 + tick_df['dy']**2)

# единичный вектор (normalized)
tick_df['dir_vec_x'] = tick_df['dx'] / length
tick_df['dir_vec_y'] = tick_df['dy'] / length

# для первой строки будет NaN, можно заменить на (0,0)
tick_df[['dir_vec_x', 'dir_vec_y']] = tick_df[['dir_vec_x', 'dir_vec_y']].fillna(0)
tick_df = tick_df.sort_values(['total_rounds_played', 'game_time']).reset_index(drop=True

In [ ]:
event_df.columns

Index(['assistedflash', 'assister_X', 'assister_Y', 'assister_name',
       'assister_steamid', 'attacker_X', 'attacker_Y', 'attacker_name',
       'attacker_steamid', 'attackerblind', 'attackerinair', 'distance',
       'dmg_armor', 'dmg_health', 'dominated', 'headshot', 'hitgroup',
       'noreplay', 'noscope', 'penetrated', 'revenge', 'thrusmoke', 'tick',
       'total_rounds_played', 'user_X', 'user_Y', 'user_name', 'user_steamid',
       'weapon', 'weapon_fauxitemid', 'weapon_itemid',
       'weapon_originalowner_xuid', 'wipe'],
      dtype='object')

In [ ]:
tick_df.columns

Index(['is_freeze_period', 'round_start_time', 'total_rounds_played',
       'is_match_started', 'round_in_progress', 'balance', 'start_balance',
       'X', 'Y', 'Z', 'tick', 'steamid', 'name'],
      dtype='object')

In [ ]:
my_mick = '...'
unique_ticks = tick_df.copy()
unique_ticks['game_time'] = unique_ticks['game_time'].apply(int)

# IF FACEIT WE SHOULD SHIFT ROUNDS by 4
unique_ticks["round_num"] = unique_ticks["total_rounds_played"] + 4

round_balances = (
    unique_ticks.groupby(["game_time", "round_num", "name", "steamid", "game_name", "map_name"])
    .agg({
        "active_weapon_name": "last",
        "team_name": "last",
        "dir_vec_x": "last", "dir_vec_y": "last",
        "is_warmup_period": "last",
        "is_alive": "last",
        "start_balance": "last",
        "balance": "last",
        "X": "last", "Y": "last", "Z": "last",
        "is_freeze_period": "last", "is_match_started": "last"
    }).reset_index()
)

round_balances[round_balances['active_weapon_name'].notna()]

,game_time,round_num,name,steamid,game_name,map_name,team_name,dir_vec_x,dir_vec_y,is_warmup_period,is_alive,start_balance,balance,X,Y,Z,is_freeze_period,is_match_started
0,187,4,AbobaValeriy,76561199366282340,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,-0.707084,-0.707129,False,True,0,0,-1662.180054,288.761993,-63.968750,True,True
1,187,4,Cimeto,76561197971275786,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,CT,0.971782,0.235882,False,True,0,0,2472.349854,2005.969971,134.505051,True,True
2,187,4,MONEYOVRBCHS,76561198055323202,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,CT,0.921829,0.387596,False,True,0,0,2353.000000,1977.000000,135.518875,True,True
3,187,4,adzuka,76561198055571210,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,CT,-0.104859,0.994487,False,True,0,0,2456.830078,2153.159912,132.077179,True,True
4,187,4,chiij,76561199400203970,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,0.996616,0.082202,False,True,0,0,-1520.060059,430.890991,-63.968750,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27885,2951,26,dADOXYZ,76561198027887817,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,0.696341,-0.717711,False,False,9200,12450,612.450378,2439.178467,241.031250,True,False
27886,2951,26,lqura,76561198213279155,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,CT,0.492538,0.870291,False,False,50,3450,1100.727783,2674.100830,132.349274,True,False
27887,2951,26,sta1evarov,76561199249362996,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,CT,0.118778,0.992921,False,False,0,3400,281.744751,2780.032715,184.299759,True,False
27888,2951,26,testicles1,76561198405927232,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,0.088201,0.996103,False,False,6800,10650,659.059082,2965.558594,144.445343,True,False


#Final Simple Dataset Creation

In [ ]:
final_dataset = round_balances[
    (round_balances['is_alive']) &
    (round_balances['start_balance'] > 0) &
    (round_balances['is_warmup_period'] == False) &
    (round_balances['active_weapon_name'].notna()) &

    (round_balances['name'] == my_mick) &
    (round_balances['round_num'] == 19)
]

# final_dataset.to_csv(f"{final_dataset['game_name'].iloc[0]}.csv")
final_dataset.head(

In [ ]:
sequences = []
from IPython.display import display
for (player_name, game_name), group in final_dataset.groupby(['name', 'game_name']):
    group = group.sort_values(['round_num', 'game_time'])
    t_seq = group[group['team_name'] == 'TERRORIST']
    ct_seq = group[group['team_name'] == 'CT']
    display(t_seq.head())
    print(f"TOTAL LENGTH, {len(t_seq)=} {len(ct_seq)=}")
    break

,game_time,round_num,name,steamid,game_name,map_name,team_name,dir_vec_x,dir_vec_y,is_warmup_period,is_alive,start_balance,balance,X,Y,Z,is_freeze_period,is_match_started
0,262,4,AbobaValeriy,76561199366282340,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,-0.707084,-0.707129,False,True,500,200,-1662.180054,288.761993,-63.96875,False,True
9,263,4,AbobaValeriy,76561199366282340,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,-0.533293,-0.845930,False,True,500,200,-1615.334839,279.762451,-63.96875,False,True
18,264,4,AbobaValeriy,76561199366282340,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,0.467806,-0.883831,False,True,500,200,-1405.148071,183.407883,-63.96875,False,True
27,265,4,AbobaValeriy,76561199366282340,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,0.285676,0.958326,False,True,500,200,-1264.673340,324.729523,-63.96875,False,True
36,266,4,AbobaValeriy,76561199366282340,85b2906f-0c54-4a38-87e4-f1880ada2701,de_inferno,TERRORIST,0.871432,0.490516,False,True,500,200,-1153.285278,508.046448,-61.96875,False,True


TOTAL LENGTH, len(t_seq)=509 len(ct_seq)=593


In [ ]:
import os
from demoparser2 import DemoParser
import numpy as np
import pandas as pd
import tqdm

def get_final_demo_dataset(demo_path):
    parser = DemoParser(demo_path)
    info = parser.parse_header()
    tick_df = parser.parse_ticks(
        ["X", "Y", "Z", "start_balance", "balance", "total_rounds_played", "is_match_started",
         "is_freeze_period", "round_in_progress", "game_time", "is_alive", "is_warmup_period", "team_name", "active_weapon_name"]
    )
    ################# FIX OF PROBLEM THAT PREV ROUND DATA APPEARS IN FUTURE ROUND BEGIN DATA
    tick_df = tick_df.sort_values(["total_rounds_played", "game_time"])

    last_freeze_rows = (
        tick_df[tick_df['is_freeze_period'] == True]
        .groupby('total_rounds_played')['game_time']
        .max()
        .reset_index()
        .rename(columns={'game_time': 'last_freeze_time'})
    )
    # 2. добавляем это время в основной датасет
    tick_df = tick_df.merge(last_freeze_rows, on='total_rounds_played', how='left')
    # 3. удаляем все строки до и включая freeze-период
    tick_df = tick_df[tick_df['game_time'] > tick_df['last_freeze_time']].drop(columns=['last_freeze_time'])

    ############## END FIX

    tick_df['map_name'] = info['map_name']
    tick_df['game_name'] = os.path.basename(demo_path).split('.')[0]

    # разница с предыдущей точкой
    tick_df['dx'] = tick_df['X'].diff()
    tick_df['dy'] = tick_df['Y'].diff()

    # длина вектора (модуль)
    length = np.sqrt(tick_df['dx']**2 + tick_df['dy']**2)

    # единичный вектор (normalized)
    tick_df['dir_vec_x'] = tick_df['dx'] / length
    tick_df['dir_vec_y'] = tick_df['dy'] / length

    # для первой строки будет NaN → (0,0)
    tick_df[['dir_vec_x', 'dir_vec_y']] = tick_df[['dir_vec_x', 'dir_vec_y']].fillna(0)
    unique_ticks = tick_df.copy()
    unique_ticks['game_time'] = unique_ticks['game_time'].apply(int)

    # IF FACEIT WE SHOULD SHIFT ROUNDS by 4
    unique_ticks["round_num"] = unique_ticks["total_rounds_played"] + 4

    round_balances = (
        unique_ticks.groupby(["game_time", "round_num", "name", "steamid", "game_name", "map_name"])
        .agg({
            "active_weapon_name": "last",
            "team_name": "last",
            "dir_vec_x": "last", "dir_vec_y": "last",
            "is_warmup_period": "last",
            "is_alive": "last",
            "start_balance": "last",
            "balance": "last",
            "X": "last", "Y": "last", "Z": "last",
            "is_freeze_period": "last", "is_match_started": "last"
        }).reset_index()
    )

    final_dataset = round_balances[
        (round_balances['is_alive']) &
        (round_balances['start_balance'] > 0) &
        (round_balances['is_warmup_period'] == False) &
        (round_balances['active_weapon_name'].notna())
    ].reset_index(drop=True)

    return final_dataset


# путь с демками
demo_dir = '/content/drive/MyDrive/DemoAI/Demos'
demos = os.listdir(demo_dir)

# собираем все датасеты
datasets = []
for demo_file in tqdm.tqdm(demos):
    demo_path = os.path.join(demo_dir, demo_file)
    try:
        df = get_final_demo_dataset(demo_path)
        datasets.append(df)
    except Exception as e:
        print(f"Ошибка при обработке {demo_file}: {e}")

# конкатенация
if datasets:
    final_dataset_concatenated = pd.concat(datasets, ignore_index=True)
    final_dataset_concatenated.to_parquet("final_dataset_updated.parquet")
    print("Финальный датасет сохранён:", final_dataset_concatenated.shape)
else:
    print("Не удалось собрать ни одного датасета.")

In [ ]:
final_dataset_concatenated.shape

(853668, 17)

# Dataloader creation

## Data loading with context (RUN FIRST)

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from tqdm import tqdm

class CustomPositionsDataset(Dataset):
    def __init__(self, parquet_path="final_dataset_updated.parquet", context_size=128, forecast_length=24):
        print("Загружаем данные...")
        self.df = pd.read_parquet(parquet_path)

        self.max_x, self.min_x = self.df['X'].max(), self.df['X'].min()
        self.max_y, self.min_y = self.df['Y'].max(), self.df['Y'].min()

        self.df['X'] = (self.df['X'] - self.min_x) / (self.max_x - self.min_x)
        self.df['Y'] = (self.df['Y'] - self.min_y) / (self.max_y - self.min_y)

        self.context_size = context_size
        self.forecast_length = forecast_length  # Новый параметр для длины прогноза

        # Сортируем данные
        self.df = self.df.sort_values(['round_num', 'game_time']).reset_index(drop=True)

        # Создаем маппинги
        self.map_to_i = {m: i for i, m in enumerate(self.df['map_name'].unique())}
        self.i_to_map = {i: m for m, i in self.map_to_i.items()}
        self.team_to_i = {t: i for i, t in enumerate(self.df['team_name'].unique())}
        self.i_to_team = {i: t for t, i in self.team_to_i.items()}
        self.weapon_to_i = {w: i for i, w in enumerate(self.df['active_weapon_name'].unique())}
        self.i_to_weapon = {i: w for i, w in self.weapon_to_i.items()}

        print("==========METADATA============")
        print("Teams:", self.team_to_i)
        print("==============================")
        print("Maps:", self.map_to_i)
        print("==============================")
        print("Weapons:", self.weapon_to_i)
        print("==============================")
        print(f"Context size: {self.context_size}, Forecast length: {self.forecast_length}")
        print("==============================")

        # Применяем маппинги
        self.df['team_id'] = self.df['team_name'].map(self.team_to_i)
        self.df['map_id'] = self.df['map_name'].map(self.map_to_i)
        self.df['weapon_id'] = self.df['active_weapon_name'].map(self.weapon_to_i)

        # Определяем колонки
        self.feature_cols = ['map_id', "weapon_id", 'round_num', 'team_id', 'X', 'Y', 'dir_vec_x', 'dir_vec_y']
        self.target_cols = ['X', 'Y', 'dir_vec_x', 'dir_vec_y']

        # Создаем последовательности
        self.sequences = []
        self._create_sequences()

    def _create_sequences(self):
        """Создает последовательности для каждого игрока в каждой игре"""
        print("Создаем последовательности...")

        # Группируем по игроку, игре И раунду
        grouped = self.df.groupby(['name', 'game_name', 'round_num'])

        for (player_name, game_name, round_num), group in tqdm(grouped, desc="Processing rounds"):
            # Сортируем по времени внутри раунда
            group = group.sort_values('game_time').reset_index(drop=True)
            
            if len(group) < self.forecast_length + 1:  # Нужно минимум forecast_length + 1 точек
                continue

            # Обрабатываем каждую команду отдельно
            for team_name in group['team_name'].unique():
                team_data = group[group['team_name'] == team_name].reset_index(drop=True)
                team_data = team_data.sort_values('game_time').reset_index(drop=True)

                if len(team_data) < self.forecast_length + 1:  # Нужно минимум forecast_length + 1 точек
                    continue

                # Создаем последовательности ТОЛЬКО внутри одного раунда
                self._create_round_sequences(team_data, player_name, game_name, team_name, round_num)

    def _create_round_sequences(self, team_data, player_name, game_name, team_name, round_num):
        """Создает последовательности для одного раунда одной команды игрока"""

        # Извлекаем фичи и таргеты
        features_data = team_data[self.feature_cols].values.astype(np.float32)
        targets_data = team_data[self.target_cols].values.astype(np.float32)

        # Проверяем временную последовательность
        times = team_data['game_time'].values
        for i in range(1, len(times)):
            if times[i] < times[i-1]:
                print(f"WARNING: Время не по порядку в раунде {round_num} для игрока {player_name}")

        # Создаем последовательности в стиле seq2seq
        # Теперь target - это последовательность из forecast_length точек
        max_start_idx = len(features_data) - self.forecast_length
        for start_idx in range(1, max_start_idx + 1):
            # Входная последовательность от начала раунда до start_idx
            input_sequence = features_data[:start_idx]  # [seq_len, 7]
            
            # Целевая последовательность - следующие forecast_length точек
            target_sequence = targets_data[start_idx:start_idx + self.forecast_length]  # [forecast_length, 4]

            # Ограничиваем длину контекста
            if len(input_sequence) > self.context_size:
                input_sequence = input_sequence[-self.context_size:]

            # Паддинг до фиксированного размера
            padded_input = self._pad_sequence(input_sequence)
            padded_target = self._pad_target_sequence(target_sequence)

            # Маска для валидных позиций входной последовательности
            input_mask = self._create_mask(len(input_sequence))
            
            # Маска для валидных позиций целевой последовательности
            target_mask = self._create_target_mask(len(target_sequence))

            self.sequences.append({
                'input': padded_input,           # [context_size, 7]
                'target': padded_target,         # [forecast_length, 4]
                'input_mask': input_mask,        # [context_size]
                'target_mask': target_mask,      # [forecast_length]
                'input_seq_length': len(input_sequence),
                'target_seq_length': len(target_sequence),
                'player': player_name,
                'game': game_name,
                'team': team_name,
                'round': round_num,
                'start_time': times[0],
                'split_time': times[start_idx],
                'end_time': times[start_idx + self.forecast_length - 1]
            })

    def _pad_sequence(self, sequence):
        """Паддинг входной последовательности до context_size"""
        seq_len = len(sequence)

        if seq_len >= self.context_size:
            return sequence[-self.context_size:]  # Берем последние context_size элементов

        # Паддинг нулями в начале
        padded = np.zeros((self.context_size, sequence.shape[1]), dtype=np.float32)
        padded[-seq_len:] = sequence  # Размещаем данные в конце

        return padded

    def _pad_target_sequence(self, target_sequence):
        """Паддинг целевой последовательности до forecast_length"""
        seq_len = len(target_sequence)
        
        if seq_len >= self.forecast_length:
            return target_sequence[:self.forecast_length]  # Берем первые forecast_length элементов
        
        # Паддинг нулями в конце
        padded = np.zeros((self.forecast_length, target_sequence.shape[1]), dtype=np.float32)
        padded[:seq_len] = target_sequence  # Размещаем данные в начале
        
        return padded

    def _create_mask(self, seq_length):
        """Создает маску для валидных позиций входной последовательности"""
        mask = np.zeros(self.context_size, dtype=bool)

        if seq_length >= self.context_size:
            mask[:] = True
        else:
            mask[-seq_length:] = True  # Валидны только последние seq_length позиций

        return mask

    def _create_target_mask(self, seq_length):
        """Создает маску для валидных позиций целевой последовательности"""
        mask = np.zeros(self.forecast_length, dtype=bool)
        mask[:seq_length] = True  # Валидны только первые seq_length позиций
        return mask

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        item = self.sequences[idx]

        return {
            'input': torch.tensor(item['input'], dtype=torch.float32),           # [context_size, 7]
            'target': torch.tensor(item['target'], dtype=torch.float32),         # [forecast_length, 4]
            'input_mask': torch.tensor(item['input_mask'], dtype=torch.bool),    # [context_size]
            'target_mask': torch.tensor(item['target_mask'], dtype=torch.bool),  # [forecast_length]
            'input_seq_length': item['input_seq_length'],
            'target_seq_length': item['target_seq_length'],
            'metadata': {
                'player': item['player'],
                'game': item['game'],
                'team': item['team'],
                'round': item['round'],
                'time_span': (item['start_time'], item['split_time'], item['end_time'])
            }
        }

    def get_stats(self):
        """Возвращает статистику датасета"""
        input_seq_lengths = [item['input_seq_length'] for item in self.sequences]
        target_seq_lengths = [item['target_seq_length'] for item in self.sequences]
        teams = [item['team'] for item in self.sequences]
        rounds = [item['round'] for item in self.sequences]

        print("\n=== СТАТИСТИКА ДАТАСЕТА ===")
        print(f"Всего последовательностей: {len(self.sequences)}")
        print("Длины входных последовательностей:")
        print(f"  Мин: {min(input_seq_lengths)}")
        print(f"  Макс: {max(input_seq_lengths)}")
        print(f"  Среднее: {np.mean(input_seq_lengths):.2f}")
        print(f"  Медиана: {np.median(input_seq_lengths):.2f}")
        
        print("Длины целевых последовательностей:")
        print(f"  Мин: {min(target_seq_lengths)}")
        print(f"  Макс: {max(target_seq_lengths)}")
        print(f"  Среднее: {np.mean(target_seq_lengths):.2f}")
        print(f"  Медиана: {np.median(target_seq_lengths):.2f}")

        # Статистика по командам
        from collections import Counter
        team_counts = Counter(teams)
        print("Распределение по командам:")
        for team, count in team_counts.items():
            print(f"  {team}: {count}")

        # Статистика по раундам
        round_counts = Counter(rounds)
        print("Распределение по раундам:")
        for round_num in sorted(round_counts.keys()):
            print(f"  Раунд {round_num}: {round_counts[round_num]}")

    def validate_sequences(self, sample_size=100):
        """Проверяет корректность созданных последовательностей"""
        print("\n=== ВАЛИДАЦИЯ ПОСЛЕДОВАТЕЛЬНОСТЕЙ ===")
        
        sample_indices = np.random.choice(len(self.sequences), 
                                        min(sample_size, len(self.sequences)), 
                                        replace=False)
        
        issues = []
        for idx in sample_indices:
            seq = self.sequences[idx]
            
            # Проверяем, что все позиции во входной последовательности в одном раунде
            rounds_in_input = set()
            # Берем только валидные данные (последние input_seq_length элементов)
            valid_input_data = seq['input'][-seq['input_seq_length']:]
            
            for pos in valid_input_data:
                rounds_in_input.add(int(pos[1]))  # round_num - второй элемент
            
            if len(rounds_in_input) > 1:
                issues.append(f"Входная последовательность {idx} содержит раунды: {rounds_in_input}")
            
            # Проверяем временную последовательность
            if seq['split_time'] >= seq['end_time']:
                issues.append(f"Последовательность {idx}: время разделения >= времени окончания")
                
            # Проверяем размеры
            if len(seq['target']) != self.forecast_length:
                issues.append(f"Последовательность {idx}: длина target {len(seq['target'])} != {self.forecast_length}")
        
        if issues:
            print(f"Найдено {len(issues)} проблем:")
            for issue in issues[:10]:  # Показываем первые 10
                print(f"  {issue}")
        else:
            print("Все проверенные последовательности корректны!")

    def get_sample_batch(self, batch_size=4):
        """Возвращает примерный batch для проверки размерностей"""
        indices = np.random.choice(len(self.sequences), min(batch_size, len(self.sequences)), replace=False)
        batch = [self[idx] for idx in indices]
        
        # Собираем batch
        inputs = torch.stack([item['input'] for item in batch])
        targets = torch.stack([item['target'] for item in batch])
        input_masks = torch.stack([item['input_mask'] for item in batch])
        target_masks = torch.stack([item['target_mask'] for item in batch])
        
        print("Размерности batch:")
        print(f"  inputs: {inputs.shape}")        # [batch_size, context_size, 7]
        print(f"  targets: {targets.shape}")      # [batch_size, forecast_length, 4]
        print(f"  input_masks: {input_masks.shape}")    # [batch_size, context_size]
        print(f"  target_masks: {target_masks.shape}")  # [batch_size, forecast_length]
        
        return {
            'inputs': inputs,
            'targets': targets,
            'input_masks': input_masks,
            'target_masks': target_masks,
            'metadata': [item['metadata'] for item in batch]
        }

# создаем датасет
dataset = CustomPositionsDataset()

# делим train/test
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
print(f"{train_size=} {val_size=} {test_size=}")
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# создаем DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=12, shuffle=True)
train_dataloader = DataLoader(
    dataset,
    batch_size=128,  # увеличить размер батча
    shuffle=True,
    pin_memory=True, # для GPU
)

Загружаем данные...
==========METADATA============
Teams: {'CT': 0, 'TERRORIST': 1}
Maps: {'de_overpass': 0, 'de_ancient': 1, 'de_dust2': 2, 'de_nuke': 3, 'de_anubis': 4, 'de_train': 5, 'de_mirage': 6, 'de_inferno': 7}
Weapons: {'Survival Knife': 0, 'USP-S': 1, 'knife_t': 2, 'Karambit': 3, 'knife': 4, 'Decoy Grenade': 5, 'C4 Explosive': 6, 'Glock-18': 7, 'P250': 8, 'Smoke Grenade': 9, 'Falchion Knife': 10, 'M9 Bayonet': 11, 'Skeleton Knife': 12, 'Butterfly Knife': 13, 'Flashbang': 14, 'High Explosive Grenade': 15, 'Dual Berettas': 16, 'Talon Knife': 17, 'Huntsman Knife': 18, 'Bayonet': 19, 'Paracord Knife': 20, 'Gut Knife': 21, 'Stiletto Knife': 22, 'Shadow Daggers': 23, 'Kukri Knife': 24, 'Five-SeveN': 25, 'Molotov': 26, 'Flip Knife': 27, 'Nomad Knife': 28, 'P2000': 29, 'Bowie Knife': 30, 'Ursus Knife': 31, 'Incendiary Grenade': 32, 'Tec-9': 33, 'Classic Knife': 34, 'Navaja Knife': 35, 'Desert Eagle': 36, 'Galil AR': 37, 'MAC-10': 38, 'MP9': 39, 'M4A1-S': 40, 'SSG 08': 41, 'MP5-SD': 4

Processing rounds: 100%|██████████| 16250/16250 [00:21<00:00, 770.29it/s]


train_size=362545 val_size=45318 test_size=45319


# Model Architecture (RUN SECOND)

In [14]:
import torch.nn as nn
class FeatureEncoder(nn.Module):
    """Универсальный энкодер для координат и направлений"""
    def __init__(self, in_dim=8, emb_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, emb_dim),
            nn.ReLU(inplace=True),  # inplace для экономии памяти
            nn.Linear(emb_dim, emb_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1)  # регуляризация
        )

    def forward(self, x):
        return self.net(x)

class TrajPredictor(nn.Module):
    def __init__(self, in_dim=8, emb_dim=512, forecast_len=24):
        super().__init__()
        # Один энкодер вместо двух одинаковых
        self.encoder = FeatureEncoder(in_dim=in_dim, emb_dim=emb_dim)
        self.forecast_len = forecast_len
        lstm_hidden_size = 512
        self.lstm = nn.LSTM(emb_dim, lstm_hidden_size, num_layers=2, batch_first=True)

        # Более эффективная голова
        self.head = nn.Sequential(
            nn.Linear(lstm_hidden_size, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 4)
        )

    def forward(self, x):
        # Один проход через энкодер
        features = self.encoder(x)                # (B, S, emb_dim)
        out, (hn, cn) = self.lstm(features)       # out: (B, S, hidden), hn: (num_layers, B, hidden)

        last_hidden = hn[-1]                 # (B, hidden)  <- берем последний слой
        decoder_input = last_hidden.unsqueeze(1)  # (B,1,hidden) -> это будет input_size == emb_dim

        preds = []
        for _ in range(self.forecast_len):
            out_step, (hn, cn) = self.lstm(decoder_input, (hn, cn))  # out_step: (B,1,hidden)
            pred = self.head(out_step.squeeze(1))                    # pred: (B, out_dim)
            preds.append(pred.unsqueeze(1))                     # соберём (B,1,out_dim)
            # на следующий шаг подаём КАК вход — сам hidden-вектор:
            decoder_input = out_step                            # (B,1,hidden)
        return torch.cat(preds, dim=1)                          # (B, F, out_dim)
        
    
    def save_checkpoint(self, path, epoch=None):
        """
        Сохранение чекпоинта модели
        
        Args:
            path: путь для сохранения
            epoch: эпоха для добавления к имени файла (опционально)
        """
        # Если указана эпоха, добавляем её к имени файла
        if epoch is not None:
            base_path, ext = os.path.splitext(path)
            if not ext:  # Если расширение не указано, добавляем .pth
                ext = '.pth'
            path = f"{base_path}_epoch_{epoch}{ext}"
        elif not os.path.splitext(path)[1]:  # Если нет расширения и нет эпохи
            path = f"{path}.pth"
        
        # Создаем директорию если её нет
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        torch.save(self.state_dict(), path)
        print(f"Model checkpoint saved to {path}")
    
    def load_checkpoint(self, path):
        """
        Загрузка чекпоинта модели
        
        Args:
            path: путь к чекпоинту
        """
        if not os.path.exists(path):
            raise FileNotFoundError(f"Checkpoint not found at {path}")
        
        self.load_state_dict(torch.load(path, map_location='cpu'))
        print(f"Model checkpoint loaded from {path}") 

# Training

In [3]:
import torch.nn.functional as F
import numpy as np
import torch

def custom_loss(pred, target, alpha=1.0, beta=1.0):
    pred_xy, target_xy = pred[:, :, :2], target[:, :, :2]       # (B, F, 2)
    pred_dir, target_dir = pred[:, :, 2:], target[:, :, 2:]     # (B, F, 2)
    
    # ADE (по координатам) - расстояние для каждого предикта
    ade_per_frame = torch.norm(pred_xy - target_xy, dim=-1)  # (B, F)
    
    # Cosine loss (по векторам направления) для каждого предикта
    cosine_loss_per_frame = 1 - F.cosine_similarity(pred_dir, target_dir, dim=-1)  # (B, F)
    
    # Комбинированный штраф для каждого предикта
    loss_per_frame = alpha * ade_per_frame + beta * cosine_loss_per_frame  # (B, F)
    
    # Суммирование по всем предиктам
    loss = loss_per_frame.sum()
    
    return loss

a, b = torch.Tensor([1, 0.8, 0.5, 0.5]).view(1, -1), torch.Tensor([0.7, 0.9, 0.4, 1]).view(1, -1)
pred_xy, target_xy = a[:, :2], b[:, :2]
pred_dir, target_dir = a[:, 2:], b[:, 2:]
torch.norm(pred_xy - target_xy, dim=1)

tensor([0.3162])

In [5]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import os

max_epochs = 25
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TrajPredictor().to(device)
print(f"Using device {device}")
val_losses = []
epoch_losses = []

def train_model_optimized(model, train_dataloader, max_epochs=200, device='cuda'):
    """Оптимизированный цикл обучения с исправленными предупреждениями"""

    # Более агрессивный оптимизатор и планировщик
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.003,
        steps_per_epoch=len(train_dataloader),
        epochs=max_epochs
    )

    # Исправленный Mixed precision для новых версий PyTorch
    scaler = torch.amp.GradScaler('cuda') if device == 'cuda' else None

    model.train()
    print(f"Using device {device}")

    for epoch in range(max_epochs):
        total_loss = 0.0
        total_samples = 0

        # Используем enumerate для более эффективного подсчета
        for batch_idx, local_data in enumerate(train_dataloader):
            local_batch = local_data['input'].to(device, non_blocking=True)
            local_labels = local_data['target'].to(device, non_blocking=True)

            optimizer.zero_grad()

            # Mixed precision forward pass с исправленным autocast
            if scaler is not None:
                with torch.amp.autocast('cuda'):
                    pred = model(local_batch)
                    loss = custom_loss(pred, local_labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()  # Перенесено после optimizer.step()
            else:
                pred = model(local_batch)
                loss = custom_loss(pred, local_labels)
                loss.backward()
                optimizer.step()
                scheduler.step()  # Перенесено после optimizer.step()

            # Аккумулируем метрики более эффективно
            batch_size = local_batch.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

            # Выводим прогресс реже для ускорения
            # if (batch_idx + 1) % 10 == 0:
            #     print(f"Epoch {epoch+1}/{max_epochs}, Batch {batch_idx+1}/{len(train_dataloader)}")
            if (batch_idx + 1) % 500 == 0:
                model.eval()
                with torch.no_grad():
                    random_batch_data = next(iter(val_dataloader))
                    random_batch, random_labels = random_batch_data['input'].to(device), random_batch_data['target'].to(device)

                    pred = model(random_batch)
                    loss = custom_loss(pred, random_labels)
                    val_losses.append(loss.item())
                    print(f"Val error {loss.item():.4f}")
                model.train()

        if epoch % 5 == 0:
            model.save_checkpoint("checkpoints/", epoch)      

        avg_loss = total_loss / total_samples
        current_lr = optimizer.param_groups[0]['lr']

        print(f"Epoch {epoch+1}/{max_epochs} — Loss: {avg_loss:.4f}, LR: {current_lr:.6f}")
        epoch_losses.append(avg_loss)

train_model_optimized(model, train_dataloader, max_epochs)

Using device cuda
Using device cuda
Val error 369.3195
Val error 401.5383
Val error 292.4144
Val error 225.8979
Val error 232.1109
Val error 144.5132
Val error 149.0042
Model checkpoint saved to checkpoints/_epoch_0.pth
Epoch 1/25 — Loss: 2531.1404, LR: 0.000245
Val error 109.8292
Val error 93.6147
Val error 82.2729
Val error 158.3597
Val error 146.6259
Val error 95.1260
Val error 89.0415
Epoch 2/25 — Loss: 1274.4960, LR: 0.000596
Val error 111.0492
Val error 96.9951
Val error 119.6486
Val error 99.8398
Val error 144.2820
Val error 139.8834
Val error 180.9106
Epoch 3/25 — Loss: 1158.6265, LR: 0.001115
Val error 177.9910
Val error 115.7791
Val error 85.8070
Val error 94.2106
Val error 111.3647
Val error 87.1476
Val error 67.3377
Epoch 4/25 — Loss: 1107.8561, LR: 0.001711
Val error 74.2866
Val error 130.7517
Val error 103.5417
Val error 120.5088
Val error 97.9000
Val error 130.6295
Val error 149.4707
Epoch 5/25 — Loss: 1081.7784, LR: 0.002280
Val error 113.8972
Val error 112.1291
Val err

# Better UI, Run after loading dataset (RUN WHEN MODEL ARCHITECTURE AND DATA DEFINED)

In [26]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox, Button, Output
import time
import os

# ================================
# КОНФИГУРАЦИЯ МАП
# ================================
MAP_CONFIGS = {
    7: {  # de_inferno
        'name': 'de_inferno',
        'image_path': r"MapCoords\INFERNO.png",
        'bounds': {'xmin': -1861, 'ymin': -833, 'xmax': 2897, 'ymax': 3660}
    },
    2: {
        'name': 'de_dust2',
        'image_path': r"MapCoords\DUST2.png",
        'bounds': {'xmin': -2186, 'ymin': -1164, 'xmax': 1788, 'ymax': 3118}
    },
    1: {
       'name': 'de_ancient',
       'image_path': r"MapCoords\ANCIENT.png",
       'bounds': {'xmin': -2299, 'ymin': -2468, 'xmax': 1396, 'ymax': 1772}
    },
    3: {
       'name': 'de_nuke',
       'image_path': r"MapCoords\NUKE_TOP.png",
       'bounds': {'xmin': -2776, 'ymin': -2480, 'xmax': 3498, 'ymax': 935}  
    },
    4: {
       'name': 'de_anubis',
       'image_path': r"MapCoords\ANUBIS.png",
       'bounds': {'xmin': -1966, 'ymin': 1745, 'xmax': 1804, 'ymax': 3157}  
    },
    5: {
       'name': 'de_train',
       'image_path': r"MapCoords\TRAIN.png",
       'bounds': {'xmin': -2177, 'ymin': -1797, 'xmax': 1797, 'ymax': 1778}  
    },
    6: {
       'name': 'de_mirage',
       'image_path': r"MapCoords\MIRAGE.png",
       'bounds': {'xmin': -2656, 'ymin': -2603, 'xmax': 1455, 'ymax': 888}  
    }
}
# Цвета для разных игроков (основной цвет для пройденного пути)
PLAYER_COLORS = [
    'red', 'darkgreen', 'darkorange', 'purple', 'brown', 
    'darkviolet', 'deeppink', 'navy', 'darkred', 'olive'
]

# ================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ================================
def calculate_prediction_error(predictions, actual_positions):
    """
    Рассчитывает среднюю ошибку предсказания (евклидово расстояние)
    predictions: список предсказанных координат
    actual_positions: список реальных координат
    """
    if not predictions or not actual_positions:
        return None
    
    # Берем минимальную длину (на случай если количество предсказаний != фактических позиций)
    min_len = min(len(predictions), len(actual_positions))
    
    errors = []
    for i in range(min_len):
        pred_x, pred_y = predictions[i]
        actual_x, actual_y = actual_positions[i]
        
        # Евклидово расстояние
        distance = np.sqrt((pred_x - actual_x)**2 + (pred_y - actual_y)**2)
        errors.append(distance)
    
    return {
        'mean_error': np.mean(errors),
        'median_error': np.median(errors),
        'max_error': np.max(errors),
        'min_error': np.min(errors),
        'std_error': np.std(errors)
    }

# ================================
# КЛАСС ДЛЯ РАБОТЫ С ВИЗУАЛИЗАЦИЕЙ
# ================================
class MapVisualizer:
    def __init__(self, map_configs):
        self.map_configs = map_configs
        self.cache = {}  # Кэш для загруженных изображений
        # Попытка загрузить шрифт
        try:
            self.font = ImageFont.truetype("arial.ttf", 12)
            self.font_small = ImageFont.truetype("arial.ttf", 10)
        except:
            self.font = ImageFont.load_default()
            self.font_small = ImageFont.load_default()
    
    def get_map_image(self, map_id):
        """Загружает изображение карты с кэшированием"""
        if map_id not in self.cache:
            config = self.map_configs.get(map_id)
            if config is None:
                raise ValueError(f"Карта с ID {map_id} не найдена в конфигурации")
            self.cache[map_id] = Image.open(config['image_path'])
        return self.cache[map_id].copy()
    
    def world_to_pixel(self, x, y, map_id):
        """Конвертирует мировые координаты в пиксельные"""
        config = self.map_configs[map_id]
        bounds = config['bounds']
        img = self.cache.get(map_id) or self.get_map_image(map_id)
        W, H = img.size
        
        scale_x = W / (bounds['xmax'] - bounds['xmin'])
        scale_y = H / (bounds['ymax'] - bounds['ymin'])
        
        px = int((x - bounds['xmin']) * scale_x)
        py = int(H - (y - bounds['ymin']) * scale_y)
        return px, py
    
    def draw_multiple_trajectories(self, trajectories_data, map_id, point_size=5):
        """
        Рисует траектории нескольких игроков на одной карте
        trajectories_data: список словарей с ключами:
            - 'original_points': список координат (красный - пройденный путь)
            - 'predictions': список предсказаний (синий)
            - 'player_name': имя игрока
            - 'color': цвет для отрисовки пройденного пути
        """
        img = self.get_map_image(map_id)
        draw = ImageDraw.Draw(img)
        
        for traj_data in trajectories_data:
            original_points = traj_data['original_points']
            predictions = traj_data['predictions']
            player_name = traj_data['player_name']
            color = traj_data['color']  # Цвет для пройденного пути
            
            # Рисуем оригинальные точки (пройденный путь) - используем цвет игрока
            for i, point in enumerate(original_points):
                px, py = self.world_to_pixel(*point, map_id)
                # Круг для точки
                draw.ellipse((px-point_size, py-point_size, 
                             px+point_size, py+point_size), 
                             fill=color, outline='black', width=1)
                
                # Отображаем ник игрока только на последней точке
                if i == len(original_points) - 1:
                    # Фон для текста
                    text_bbox = draw.textbbox((px+10, py-10), player_name, font=self.font)
                    draw.rectangle(text_bbox, fill='white', outline=color, width=1)
                    draw.text((px+10, py-10), player_name, fill=color, font=self.font)
            
            # Рисуем линию траектории пройденного пути
            if len(original_points) > 1:
                for i in range(len(original_points) - 1):
                    px1, py1 = self.world_to_pixel(*original_points[i], map_id)
                    px2, py2 = self.world_to_pixel(*original_points[i+1], map_id)
                    draw.line([(px1, py1), (px2, py2)], fill=color, width=2)
            
            # Рисуем предсказания СИНИМ ЦВЕТОМ
            if predictions:
                # Соединяем последнюю оригинальную точку с первым предсказанием
                if original_points:
                    px_last, py_last = self.world_to_pixel(*original_points[-1], map_id)
                    px_pred, py_pred = self.world_to_pixel(*predictions[0], map_id)
                    # Пунктирная линия синего цвета
                    self._draw_dashed_line(draw, px_last, py_last, px_pred, py_pred, 'blue')
                
                # Рисуем точки предсказаний СИНИМ
                for i, prediction in enumerate(predictions):
                    px_pred, py_pred = self.world_to_pixel(*prediction, map_id)
                    # Меньший размер для предсказаний
                    pred_size = point_size - 2
                    draw.ellipse((px_pred-pred_size, py_pred-pred_size, 
                                 px_pred+pred_size, py_pred+pred_size), 
                                 fill='blue', outline='white', width=1)
                    
                    # Пунктирные линии между предсказаниями (синие)
                    if i < len(predictions) - 1:
                        px_next, py_next = self.world_to_pixel(*predictions[i+1], map_id)
                        self._draw_dashed_line(draw, px_pred, py_pred, px_next, py_next, 'blue')
        
        return img
    
    def _draw_dashed_line(self, draw, x1, y1, x2, y2, color, dash_length=5):
        """Рисует пунктирную линию"""
        import math
        dx = x2 - x1
        dy = y2 - y1
        distance = math.sqrt(dx**2 + dy**2)
        
        if distance == 0:
            return
        
        dashes = int(distance / dash_length)
        for i in range(0, dashes, 2):
            start = i / dashes
            end = min((i + 1) / dashes, 1)
            sx = x1 + dx * start
            sy = y1 + dy * start
            ex = x1 + dx * end
            ey = y1 + dy * end
            draw.line([(sx, sy), (ex, ey)], fill=color, width=2)

# ================================
# КЛАСС ДЛЯ ИНДЕКСАЦИИ И ФИЛЬТРАЦИИ ДАННЫХ
# ================================
class DataIndexer:
    def __init__(self, test_dataloader, dataset, min_length=50):
        self.dataloader = test_dataloader
        self.dataset = dataset
        self.min_length = min_length
        self.index = None
        self.batches_cache = {}  # Кэш для батчей
        print(f"Создаем индекс данных с минимальной длиной {min_length}...")
        self._build_index()
    
    def update_min_length(self, new_min_length):
        """Обновляет минимальную длину и перестраивает индекс"""
        self.min_length = new_min_length
        print(f"Обновляем индекс с минимальной длиной {new_min_length}...")
        self._build_index()
    
    def _build_index(self):
        """Строит индекс всех доступных данных"""
        index_data = []
        
        for batch_idx, batch in enumerate(tqdm(self.dataloader, desc="Индексация")):
            # Кэшируем батч
            self.batches_cache[batch_idx] = batch
            
            if isinstance(batch, (tuple, list)):
                inputs = batch[0]
            elif isinstance(batch, dict):
                inputs = batch['input']
            else:
                inputs = batch
            
            metadata = batch.get('metadata', {})
            
            for i in range(inputs.shape[0]):
                arr = inputs[i]
                if hasattr(arr, 'cpu'):
                    arr = arr.cpu().numpy()
                
                mask = ~(np.all(arr == 0, axis=1))
                filtered = arr[mask]
                
                if filtered.shape[0] >= self.min_length:
                    map_id = int(filtered[-1][0])
                    round_num = int(filtered[-1][2])
                    team_id = int(filtered[-1][3])
                    
                    # Получаем метаданные
                    player = metadata.get('player', [None] * inputs.shape[0])[i] if 'player' in metadata else f'Player_{i}'
                    team_name = metadata.get('team', [None] * inputs.shape[0])[i] if 'team' in metadata else None
                    game = metadata.get('game', [None] * inputs.shape[0])[i] if 'game' in metadata else f'Game_{batch_idx}'
                    
                    index_data.append({
                        'batch_idx': batch_idx,
                        'sample_idx': i,
                        'map_id': map_id,
                        'map_name': self.dataset.i_to_map.get(map_id, f'Map_{map_id}'),
                        'round_num': round_num,
                        'team_id': team_id,
                        'team_name': team_name,
                        'player': player,
                        'game': game,
                        'length': filtered.shape[0]
                    })
        
        self.index = pd.DataFrame(index_data)
        print(f"Индекс создан: {len(self.index)} записей")
        print(f"Доступные карты: {self.index['map_name'].unique()}")
        print(f"Доступные игры: {self.index['game'].nunique()}")
        print(f"Диапазон длин раундов: {self.index['length'].min()} - {self.index['length'].max()}")
    
    def get_available_maps(self):
        """Возвращает список доступных карт"""
        return sorted(self.index[['map_id', 'map_name']].drop_duplicates().values.tolist())
    
    def get_games_for_map(self, map_id):
        """Возвращает список игр для выбранной карты"""
        return sorted(self.index[self.index['map_id'] == map_id]['game'].unique().tolist())
    
    def get_rounds_for_game(self, map_id, game):
        """Возвращает список раундов для выбранной игры"""
        mask = (self.index['map_id'] == map_id) & (self.index['game'] == game)
        return sorted(self.index[mask]['round_num'].unique().tolist())
    
    def get_players_for_round(self, map_id, game, round_num):
        """Возвращает список уникальных игроков в раунде с агрегированными данными"""
        mask = (self.index['map_id'] == map_id) & \
               (self.index['game'] == game) & \
               (self.index['round_num'] == round_num)
        
        round_data = self.index[mask]
        
        # Группируем по игроку и берем запись с максимальной длиной для каждого игрока
        unique_players = []
        for player_name in round_data['player'].unique():
            player_records = round_data[round_data['player'] == player_name]
            # Берем запись с максимальной длиной (самая полная траектория)
            best_record = player_records.loc[player_records['length'].idxmax()]
            unique_players.append({
                'player': best_record['player'],
                'team_name': best_record['team_name'],
                'team_id': best_record['team_id'],
                'length': best_record['length'],
                'batch_idx': best_record['batch_idx'],
                'sample_idx': best_record['sample_idx']
            })
        
        # Сортируем по команде и имени
        unique_players.sort(key=lambda x: (x['team_id'], x['player']))
        
        return unique_players
    
    def get_data(self, map_id, game, round_num, players=None, team_filter=None):
        """Получает данные по фильтрам (возвращает уникальные записи для каждого игрока)"""
        mask = (self.index['map_id'] == map_id) & \
               (self.index['game'] == game) & \
               (self.index['round_num'] == round_num)
        
        if team_filter == 'CT':
            mask &= self.index['team_id'] == 1  # Предполагаем CT=1
        elif team_filter == 'T':
            mask &= self.index['team_id'] == 0  # Предполагаем T=0
        
        if players:
            mask &= self.index['player'].isin(players)
        
        round_data = self.index[mask]
        
        # Группируем по игроку и берем запись с максимальной длиной для каждого игрока
        unique_records = []
        for player_name in round_data['player'].unique():
            player_records = round_data[round_data['player'] == player_name]
            # Берем запись с максимальной длиной (самая полная траектория)
            best_record = player_records.loc[player_records['length'].idxmax()]
            unique_records.append(best_record.to_dict())
        
        return unique_records
    
    def get_batch_data(self, batch_idx, sample_idx):
        """Получает данные из кэша батчей"""
        batch = self.batches_cache.get(batch_idx)
        if batch is None:
            raise ValueError(f"Батч {batch_idx} не найден в кэше")
        
        if isinstance(batch, (tuple, list)):
            inputs = batch[0]
        elif isinstance(batch, dict):
            inputs = batch['input']
        else:
            inputs = batch
        
        arr = inputs[sample_idx]
        if hasattr(arr, 'cpu'):
            arr = arr.cpu().numpy()
        
        mask = ~(np.all(arr == 0, axis=1))
        filtered = arr[mask]
        
        return filtered

# ================================
# ОСНОВНОЙ КЛАСС ПРИЛОЖЕНИЯ
# ================================
class TrajectoryExplorer:
    def __init__(self, model, test_dataloader, dataset, map_configs, initial_min_length=50):
        self.model = model
        self.dataloader = test_dataloader
        self.dataset = dataset
        self.visualizer = MapVisualizer(map_configs)
        self.indexer = DataIndexer(test_dataloader, dataset, initial_min_length)
        
        # Для ручного режима
        self.manual_mode_active = False
        self.show_all_prediction_path = False
        self.current_step = 0
        self.players_data_cache = []
        self.max_steps = 0
        
        # UI элементы
        self.output = Output()
        self.setup_ui()
    
    def setup_ui(self):
        """Создает интерфейс"""
        # Минимальная длина раунда
        self.min_length_slider = widgets.IntSlider(
            value=self.indexer.min_length,
            min=10,
            max=200,
            step=5,
            description='Мин. длина:',
            style={'description_width': '100px'},
            continuous_update=False
        )
        self.min_length_slider.observe(self.on_min_length_change, names='value')
        
        # Кнопка обновления индекса
        self.rebuild_index_btn = Button(
            description='Обновить индекс',
            button_style='info',
            icon='refresh',
            layout=widgets.Layout(width='150px')
        )
        self.rebuild_index_btn.on_click(self.on_rebuild_index_click)
        
        # Выбор карты
        map_options = [(f"{name} (ID: {id})", id) 
                       for id, name in self.indexer.get_available_maps()]
        self.map_dropdown = widgets.Dropdown(
            options=map_options,
            description='Карта:',
            style={'description_width': '100px'}
        )
        
        # Выбор игры
        self.game_dropdown = widgets.Dropdown(
            options=[],
            description='Игра:',
            style={'description_width': '100px'}
        )
        
        # Выбор раунда
        self.round_dropdown = widgets.Dropdown(
            options=[],
            description='Раунд:',
            style={'description_width': '100px'}
        )
        
        # Фильтр по команде
        self.team_radio = widgets.RadioButtons(
            options=['Все', 'CT', 'T'],
            description='Команда:',
            style={'description_width': '100px'}
        )
        self.team_radio.observe(self.on_team_change, names='value')
        
        # Чекбоксы для выбора игроков
        self.player_checkboxes_container = VBox([])
        
        # Кнопка "Выбрать всех"
        self.select_all_btn = Button(
            description='Выбрать всех',
            button_style='',
            icon='check',
            layout=widgets.Layout(width='120px')
        )
        self.select_all_btn.on_click(self.on_select_all_click)
        
        # Кнопка "Снять все"
        self.deselect_all_btn = Button(
            description='Снять все',
            button_style='',
            icon='times',
            layout=widgets.Layout(width='120px')
        )
        self.deselect_all_btn.on_click(self.on_deselect_all_click)

        # Чекбокс "Показывать весь путь предсказаний? (Шумный)"
        self.show_full_path_checkbox = widgets.Checkbox(
            value=False,
            description='Показывать весь путь предсказаний? (Может быть шумным)',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )

        # Кнопка визуализации
        self.visualize_btn = Button(
            description='Визуализировать',
            button_style='success',
            icon='play',
            layout=widgets.Layout(width='150px')
        )
        self.visualize_btn.on_click(self.on_visualize_click)
        
        # Настройки визуализации - теперь с выбором режима
        self.mode_radio = widgets.RadioButtons(
            options=['Статичный', 'Анимация', 'Ручной'],
            value='Статичный',
            description='Режим:',
            style={'description_width': '100px'}
        )
        self.mode_radio.observe(self.on_mode_change, names='value')
        
        self.speed_slider = widgets.FloatSlider(
            value=0.5,
            min=0.1,
            max=5,
            step=0.1,
            description='Скорость:',
            style={'description_width': '100px'},
            disabled=True  # Отключен по умолчанию
        )
        
        self.point_size_slider = widgets.IntSlider(
            value=5,
            min=2,
            max=10,
            step=1,
            description='Размер точек:',
            style={'description_width': '100px'}
        )
        
        # Кнопки управления для ручного режима
        self.prev_step_btn = Button(
            description='◀ Назад',
            button_style='primary',
            icon='arrow-left',
            layout=widgets.Layout(width='120px'),
            disabled=True
        )
        self.prev_step_btn.on_click(self.on_prev_step_click)
        
        self.next_step_btn = Button(
            description='Вперед ▶',
            button_style='primary',
            icon='arrow-right',
            layout=widgets.Layout(width='120px'),
            disabled=True
        )
        self.next_step_btn.on_click(self.on_next_step_click)
        
        self.step_label = widgets.HTML(
            value="<b>Шаг: 0/0</b>",
            layout=widgets.Layout(width='150px')
        )
        
        # Обработчики изменений
        self.map_dropdown.observe(self.on_map_change, names='value')
        self.game_dropdown.observe(self.on_game_change, names='value')
        self.round_dropdown.observe(self.on_round_change, names='value')
        
        # Инициализация
        if map_options:
            self.on_map_change({'new': map_options[0][1]})

    def on_mode_change(self, change):
        """Обработчик изменения режима визуализации"""
        mode = change['new']
        
        # Управление доступностью элементов
        if mode == 'Анимация':
            self.speed_slider.disabled = False
            self.prev_step_btn.disabled = True
            self.next_step_btn.disabled = True
        elif mode == 'Ручной':
            self.speed_slider.disabled = True
            self.prev_step_btn.disabled = False
            self.next_step_btn.disabled = False
        else:  # Статичный
            self.speed_slider.disabled = True
            self.prev_step_btn.disabled = True
            self.next_step_btn.disabled = True
    
    def on_prev_step_click(self, btn):
        """Шаг назад в ручном режиме"""
        if self.manual_mode_active and self.current_step > 1:
            self.current_step -= 1
            self.render_manual_step()
    
    def on_next_step_click(self, btn):
        """Шаг вперед в ручном режиме"""
        if self.manual_mode_active and self.current_step < self.max_steps:
            self.current_step += 1
            self.render_manual_step()
    
    def render_manual_step(self):
        """Отрисовка текущего шага в ручном режиме"""
        with self.output:
            clear_output(wait=True)
            
            map_id = self.map_dropdown.value
            j = self.current_step
            
            trajectories_data = []
            current_errors = []
            
            for player_data in self.players_data_cache:
                filtered = player_data['filtered']
                pairs = player_data['pairs']
                
                if j < len(filtered) - 24:
                    # Делаем предсказание для текущего шага
                    window = np.zeros((128, filtered.shape[1]))
                    seq_len = min(j, 128)
                    window[-seq_len:] = filtered[j-seq_len:j]
                    
                    with torch.no_grad():
                        window_tensor = torch.tensor(window, dtype=torch.float32).unsqueeze(0).to('cuda')
                        pred = self.model(window_tensor)
                        pred_coords = pred.squeeze().cpu().numpy()[:, :2]
                        predictions = [tuple(coords) for coords in pred_coords]
                    
                    scaled_predictions = self.scale_coordinates(predictions)
                    
                    # Если галочка не стоит - берем только последнюю точку
                    if not self.show_full_path_checkbox.value:
                        scaled_predictions = [scaled_predictions[-1]] if scaled_predictions else []
                    
                    # Рассчитываем ошибку для текущего шага
                    future_actual = pairs[j:j+24]
                    future_scaled = self.scale_coordinates(future_actual)
                    error_stats = calculate_prediction_error(
                        self.scale_coordinates(predictions),  # Для расчета используем весь путь
                        future_scaled
                    )
                    
                    if error_stats:
                        current_errors.append({
                            'player': player_data['player_name'],
                            'color': player_data['color'],
                            'mean_error': error_stats['mean_error']
                        })
                else:
                    scaled_predictions = [] 
                
                scaled_pairs = self.scale_coordinates(pairs[:min(j + 1, len(pairs))])
                
                trajectories_data.append({
                    'original_points': scaled_pairs,
                    'predictions': scaled_predictions,
                    'player_name': player_data['player_name'],
                    'color': player_data['color']
                })
            
            img = self.visualizer.draw_multiple_trajectories(
                trajectories_data, 
                map_id,
                point_size=self.point_size_slider.value
            )
            display(img)
            
            # Обновляем метку шага
            self.step_label.value = f"<b>Шаг: {j}/{self.max_steps}</b>"
            
            # Выводим информацию
            print("\n" + "="*60)
            print(f"⏱️  ШАГ: {j}/{self.max_steps}")
            print("="*60)
            
            print("\n📊 ЛЕГЕНДА:")
            for traj in trajectories_data:
                print(f"  🎨 {traj['color'].upper()}: {traj['player_name']}")
            
            if current_errors:
                print("\n🎯 ТОЧНОСТЬ ПРЕДСКАЗАНИЙ (текущий шаг):")
                for error_info in current_errors:
                    print(f"  👤 {error_info['player']} ({error_info['color'].upper()}): {error_info['mean_error']:.2f} единиц")
                
                avg_error = np.mean([e['mean_error'] for e in current_errors])
                print(f"\n  🌟 Средняя ошибка: {avg_error:.2f} единиц")
            
            print("="*60)
    
    def on_min_length_change(self, change):
        """Обработчик изменения минимальной длины"""
        pass
    
    def on_rebuild_index_click(self, btn):
        """Обработчик кнопки обновления индекса"""
        with self.output:
            clear_output(wait=True)
            new_min_length = self.min_length_slider.value
            self.indexer.update_min_length(new_min_length)
            
            # Обновляем список карт
            map_options = [(f"{name} (ID: {id})", id) 
                           for id, name in self.indexer.get_available_maps()]
            self.map_dropdown.options = map_options
            if map_options:
                self.on_map_change({'new': map_options[0][1]})
            
            print("✓ Индекс обновлен!")
    
    def on_map_change(self, change):
        """Обработчик изменения карты"""
        map_id = change['new']
        games = self.indexer.get_games_for_map(map_id)
        self.game_dropdown.options = games
        if games:
            self.game_dropdown.value = games[0]
    
    def on_game_change(self, change):
        """Обработчик изменения игры"""
        game = change['new']
        map_id = self.map_dropdown.value
        rounds = self.indexer.get_rounds_for_game(map_id, game)
        self.round_dropdown.options = rounds
        if rounds:
            self.round_dropdown.value = rounds[0]
    
    def on_round_change(self, change):
        """Обработчик изменения раунда"""
        self.update_player_checkboxes()
    
    def on_team_change(self, change):
        """Обработчик изменения команды"""
        self.update_player_checkboxes()
    
    def update_player_checkboxes(self):
        """Обновляет чекбоксы игроков"""
        round_num = self.round_dropdown.value
        map_id = self.map_dropdown.value
        game = self.game_dropdown.value
        team_filter = self.team_radio.value if self.team_radio.value != 'Все' else None
        
        players_data = self.indexer.get_players_for_round(map_id, game, round_num)
        
        # Фильтруем по команде если нужно
        if team_filter == 'CT':
            players_data = [p for p in players_data if p['team_id'] == 1]
        elif team_filter == 'T':
            players_data = [p for p in players_data if p['team_id'] == 0]
        
        # Создаем чекбоксы
        checkboxes = []
        for player_data in players_data:
            checkbox = widgets.Checkbox(
                value=False,
                description=f"{player_data['player']} ({player_data['team_name']}, len={player_data['length']})",
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='400px')
            )
            checkbox.player_data = player_data  # Сохраняем данные игрока
            checkboxes.append(checkbox)
        
        self.player_checkboxes_container.children = checkboxes
    
    def on_select_all_click(self, btn):
        """Выбрать всех игроков"""
        for checkbox in self.player_checkboxes_container.children:
            checkbox.value = True
    
    def on_deselect_all_click(self, btn):
        """Снять выбор со всех игроков"""
        for checkbox in self.player_checkboxes_container.children:
            checkbox.value = False
    
    def on_visualize_click(self, btn):
        """Обработчик кнопки визуализации"""
        with self.output:
            clear_output(wait=True)
            
            # Получаем выбранных игроков
            selected_records = []
            for checkbox in self.player_checkboxes_container.children:
                if checkbox.value:
                    selected_records.append(checkbox.player_data)
            
            if not selected_records:
                print("❌ Выберите хотя бы одного игрока")
                return
            
            map_id = self.map_dropdown.value
            
            print(f"✓ Выбрано {len(selected_records)} уникальных игроков")
            
            mode = self.mode_radio.value
            
            if mode == 'Анимация':
                # Анимированная визуализация
                self.manual_mode_active = False
                self.animate_multiple_trajectories(selected_records, map_id)
            elif mode == 'Ручной':
                # Ручной режим
                self.start_manual_mode(selected_records, map_id)
            else:
                # Статичная визуализация
                self.manual_mode_active = False
                self.visualize_multiple_trajectories(selected_records, map_id)
    
    def start_manual_mode(self, records, map_id):
        """Запускает ручной режим управления"""
        # Подготавливаем данные для всех игроков
        self.players_data_cache = []
        self.max_steps = 0
        
        for idx, record in enumerate(records):
            filtered = self.indexer.get_batch_data(record['batch_idx'], record['sample_idx'])
            X = filtered[:, [4, 5]]
            pairs = [tuple(row) for row in X.tolist()]
            
            color = PLAYER_COLORS[idx % len(PLAYER_COLORS)]
            
            self.players_data_cache.append({
                'filtered': filtered,
                'pairs': pairs,
                'player_name': record['player'],
                'color': color
            })
            
            self.max_steps = max(self.max_steps, len(filtered) - 24)
        
        # Устанавливаем начальный шаг
        self.current_step = 1
        self.manual_mode_active = True
        
        # Отрисовываем первый шаг
        self.render_manual_step()
    
    def visualize_multiple_trajectories(self, records, map_id):
        """Визуализирует траектории нескольких игроков одновременно"""
        trajectories_data = []
        all_errors = []
        
        for idx, record in enumerate(records):
            # Получаем данные
            filtered = self.indexer.get_batch_data(record['batch_idx'], record['sample_idx'])
            
            # Получаем координаты
            X = filtered[:, [4, 5]]
            pairs = [tuple(row) for row in X.tolist()]
            
            # Получаем предсказания
            predictions = self.get_full_predictions(filtered)
            
            # Масштабируем координаты
            scaled_pairs = self.scale_coordinates(pairs)
            scaled_predictions = self.scale_coordinates(predictions)
            
            # Рассчитываем ошибку (сравниваем с будущими 24 шагами)
            if len(pairs) > len(filtered) - 24:
                # Если есть реальные будущие позиции для сравнения
                future_actual = pairs[len(pairs)-24:]
                future_scaled = self.scale_coordinates(future_actual)
                error_stats = calculate_prediction_error(scaled_predictions, future_scaled)
                if error_stats:
                    all_errors.append({
                        'player': record['player'],
                        'stats': error_stats
                    })
            
            # Выбираем цвет
            color = PLAYER_COLORS[idx % len(PLAYER_COLORS)]
            
            trajectories_data.append({
                'original_points': scaled_pairs,
                'predictions': scaled_predictions,
                'player_name': record['player'],
                'color': color
            })
        
        # Рисуем все траектории на одной карте
        img = self.visualizer.draw_multiple_trajectories(
            trajectories_data, 
            map_id, 
            point_size=self.point_size_slider.value
        )
        display(img)
        
        # Выводим статистику
        print("\n" + "="*60)
        print("📊 ЛЕГЕНДА И СТАТИСТИКА")
        print("="*60)
        
        for traj in trajectories_data:
            print(f"\n🎨 {traj['color'].upper()}: {traj['player_name']}")
        
        if all_errors:
            print("\n" + "="*60)
            print("🎯 ТОЧНОСТЬ ПРЕДСКАЗАНИЙ")
            print("="*60)
            
            for error_info in all_errors:
                stats = error_info['stats']
                print(f"\n👤 {error_info['player']}:")
                print(f"   📏 Средняя ошибка: {stats['mean_error']:.2f} единиц")
                print(f"   📊 Медианная ошибка: {stats['median_error']:.2f} единиц")
                print(f"   ⬆️  Макс. ошибка: {stats['max_error']:.2f} единиц")
                print(f"   ⬇️  Мин. ошибка: {stats['min_error']:.2f} единиц")
                print(f"   📈 Стд. отклонение: {stats['std_error']:.2f} единиц")
            
            # Общая статистика
            all_mean_errors = [e['stats']['mean_error'] for e in all_errors]
            print(f"\n{'='*60}")
            print(f"🌟 ОБЩАЯ СРЕДНЯЯ ОШИБКА: {np.mean(all_mean_errors):.2f} единиц")
            print(f"{'='*60}")
    
    def animate_multiple_trajectories(self, records, map_id):
        """Анимирует траектории нескольких игроков одновременно"""
        # Подготавливаем данные для всех игроков
        players_data = []
        max_length = 0
        
        for idx, record in enumerate(records):
            filtered = self.indexer.get_batch_data(record['batch_idx'], record['sample_idx'])
            X = filtered[:, [4, 5]]
            pairs = [tuple(row) for row in X.tolist()]
            
            color = PLAYER_COLORS[idx % len(PLAYER_COLORS)]
            
            players_data.append({
                'filtered': filtered,
                'pairs': pairs,
                'player_name': record['player'],
                'color': color,
                'predictions': [],
                'errors': []
            })
            
            max_length = max(max_length, len(filtered))
        
        # Анимация
        for j in range(1, max_length - 24 + 1):
            trajectories_data = []
            current_errors = []
            
            for player_data in players_data:
                filtered = player_data['filtered']
                pairs = player_data['pairs']
                
                if j < len(filtered) - 24:
                    # Делаем предсказание для текущего шага
                    window = np.zeros((128, filtered.shape[1]))
                    seq_len = min(j, 128)
                    window[-seq_len:] = filtered[j-seq_len:j]
                    
                    with torch.no_grad():
                        window_tensor = torch.tensor(window, dtype=torch.float32).unsqueeze(0).to('cuda')
                        pred = self.model(window_tensor)
                        pred_coords = pred.squeeze().cpu().numpy()[:, :2]
                        predictions = [tuple(coords) for coords in pred_coords]
                    
                    scaled_predictions = self.scale_coordinates(predictions)
                    
                    # Если галочка не стоит - берем только последнюю точку
                    if not self.show_full_path_checkbox.value:
                        scaled_predictions = [scaled_predictions[-1]] if scaled_predictions else []
                    
                    # Рассчитываем ошибку для текущего шага
                    future_actual = pairs[j:j+24]
                    future_scaled = self.scale_coordinates(future_actual)
                    error_stats = calculate_prediction_error(
                        self.scale_coordinates(predictions),  # Для расчета используем весь путь
                        future_scaled
                    )
                    
                    if error_stats:
                        current_errors.append({
                            'player': player_data['player_name'],
                            'color': player_data['color'],
                            'mean_error': error_stats['mean_error']
                        })
                else:
                    scaled_predictions = []
                
                scaled_pairs = self.scale_coordinates(pairs[:min(j + 1, len(pairs))])
                
                trajectories_data.append({
                    'original_points': scaled_pairs,
                    'predictions': scaled_predictions,
                    'player_name': player_data['player_name'],
                    'color': player_data['color']
                })
            
            clear_output(wait=True)
            img = self.visualizer.draw_multiple_trajectories(
                trajectories_data, 
                map_id,
                point_size=self.point_size_slider.value
            )
            display(img)
            
            # Выводим информацию
            print("\n" + "="*60)
            print(f"⏱️  ШАГ: {j}/{max_length-24}")
            print("="*60)
            
            print("\n📊 ЛЕГЕНДА:")
            for traj in trajectories_data:
                print(f"  🎨 {traj['color'].upper()}: {traj['player_name']}")
            
            if current_errors:
                print("\n🎯 ТОЧНОСТЬ ПРЕДСКАЗАНИЙ (текущий шаг):")
                for error_info in current_errors:
                    print(f"  👤 {error_info['player']} ({error_info['color'].upper()}): {error_info['mean_error']:.2f} единиц")
                
                avg_error = np.mean([e['mean_error'] for e in current_errors])
                print(f"\n  🌟 Средняя ошибка: {avg_error:.2f} единиц")
            
            print("="*60)
            
            time.sleep(self.speed_slider.value)
    
    def get_full_predictions(self, filtered):
        """Получает все предсказания для полной последовательности"""
        window = np.zeros((128, filtered.shape[1]))
        seq_len = min(len(filtered), 128)
        window[-seq_len:] = filtered[-seq_len:]
        
        with torch.no_grad():
            window_tensor = torch.tensor(window, dtype=torch.float32).unsqueeze(0).to('cuda')
            pred = self.model(window_tensor)
            pred_coords = pred.squeeze().cpu().numpy()[:, :2]
            predictions = [tuple(coords) for coords in pred_coords]
        
        return predictions
    
    def scale_coordinates(self, coords):
        """Масштабирует координаты обратно в мировые"""
        return [(x * (self.dataset.max_x - self.dataset.min_x) + self.dataset.min_x,
                 y * (self.dataset.max_y - self.dataset.min_y) + self.dataset.min_y)
                for x, y in coords]
    
    def display(self):
        """Отображает интерфейс"""
        # Секция настроек индекса
        index_section = VBox([
            widgets.HTML(value="<b>⚙️ Настройки индекса</b>"),
            HBox([self.min_length_slider, self.rebuild_index_btn])
        ])
        
        # Секция фильтров
        filter_section = VBox([
            widgets.HTML(value="<b>🔍 Фильтры</b>"),
            HBox([self.map_dropdown, self.game_dropdown, self.round_dropdown]),
            self.team_radio
        ])
        
        # Секция выбора игроков
        player_section = VBox([
            widgets.HTML(value="<b>👥 Выбор игроков</b>"),
            HBox([self.select_all_btn, self.deselect_all_btn]),
            widgets.HTML(value="<div style='max-height: 300px; overflow-y: auto;'>"),
            self.player_checkboxes_container,
        ])
        
        # Секция визуализации
        viz_section = VBox([
            widgets.HTML(value="<b>🎨 Настройки визуализации</b>"),
            self.show_full_path_checkbox,  # <- Добавь эту строку
            self.mode_radio,
            HBox([self.speed_slider, self.point_size_slider]),
            HBox([self.prev_step_btn, self.step_label, self.next_step_btn]),
            self.visualize_btn
        ])
        
        controls = VBox([
            index_section,
            widgets.HTML(value="<hr>"),
            filter_section,
            widgets.HTML(value="<hr>"),
            player_section,
            widgets.HTML(value="<hr>"),
            viz_section
        ])
        
        return VBox([controls, self.output])

# ================================
# ИСПОЛЬЗОВАНИЕ
# ================================

# Загрузка модели
model = TrajPredictor().to('cuda')
model.load_checkpoint(r"G:\DemoAI\checkpoints\_epoch_20.pth")
model.eval()

# Создание приложения с начальной минимальной длиной 50
app = TrajectoryExplorer(model, test_dataloader, dataset, MAP_CONFIGS, initial_min_length=50)

# Отображение интерфейса
display(app.display())

Model checkpoint loaded from G:\DemoAI\checkpoints\_epoch_20.pth
Создаем индекс данных с минимальной длиной 50...


Индексация:   0%|          | 0/709 [00:00<?, ?it/s]

Индекс создан: 7176 записей
Доступные карты: ['de_dust2' 'de_overpass' 'de_anubis' 'de_inferno' 'de_train' 'de_mirage'
 'de_nuke' 'de_ancient']
Доступные игры: 80
Диапазон длин раундов: 50 - 128
